In [13]:
import sys
import importlib
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import sys
import awkward as ak
import dask
from dask.distributed import Client
import math
sys.path.insert(0,"../")

In [2]:
sys.path.insert(0,"../python/HNL_Plotting_HelperFunctions")
import MuonSystemReader

/uscms/home/amalbert/.local/lib/python3.9/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/cvmfs/cms.cern.ch/el8_amd64_gcc12/external/py3-numpy/1.24.3-0831476b9e4ddf0b4b9eb1f4c971c0d1/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])

In [3]:
HNL_2GeV_10ctau_path = "root://cmseos.fnal.gov//store/group/lpclonglived/amalbert/HNL_Tau_Search/signal_tau_mN2_ct1000/HNL_tau_mN_2_ctau_1000/normalized/HNL_tau_mN_2_ctau_1000.root"
signal_events  = MuonSystemReader.loadTree_nanoFactory(HNL_2GeV_10ctau_path)

/uscms/home/amalbert/.local/lib/python3.9/site-packages/dask_awkward/lib/structure.py:911: UserWarning: Please ensure that dask.awkward<nCscRechitClusters, npartitions=1>
        is partitionwise-compatible with dask.awkward<getitem, npartitions=1>
        (e.g. counts comes from a dak.num(array, axis=1)),
        otherwise this unflatten operation will fail when computed!
  warnings.warn(


In [4]:
### NOTE: for this sample in data, the tau energy branch is messed up ###
data_path_base = "root://cmseos.fnal.gov//store/group/lpclonglived/amalbert/HNL_Tau_Search/2024_Data_allChannels/"

data_samples = ["Muon0-Run2024B-PromptReco-v1",
    "Muon0-Run2024C-PromptReco-v1",
    "Muon0-Run2024D-PromptReco-v1",
    "Muon0-Run2024E-PromptReco-v1",
    "Muon0-Run2024E-PromptReco-v2",
    "Muon0-Run2024F-PromptReco-v1",
    "Muon0-Run2024G-PromptReco-v1",
    "Muon0-Run2024H-PromptReco-v1",
    "Muon0-Run2024I-PromptReco-v1",
    "Muon0-Run2024I-PromptReco-v2",
    "Muon1-Run2024B-PromptReco-v1",
    "Muon1-Run2024C-PromptReco-v1",
    "Muon1-Run2024D-PromptReco-v1",
    "Muon1-Run2024E-PromptReco-v1",
    "Muon1-Run2024E-PromptReco-v2",
    "Muon1-Run2024F-PromptReco-v1",
    "Muon1-Run2024G-PromptReco-v1",
    "Muon1-Run2024H-PromptReco-v1",
    "Muon1-Run2024I-PromptReco-v1",
    "Muon1-Run2024I-PromptReco-v2"
    ]

data_events_list = [data_path_base+sample+"/normalized/"+sample+"_goodLumi.root" for sample in data_samples]
data_events  = MuonSystemReader.loadTree_nanoFactory(data_events_list)


/uscms/home/amalbert/.local/lib/python3.9/site-packages/dask_awkward/lib/structure.py:911: UserWarning: Please ensure that dask.awkward<nCscRechitClusters, npartitions=20>
        is partitionwise-compatible with dask.awkward<getitem, npartitions=20>
        (e.g. counts comes from a dak.num(array, axis=1)),
        otherwise this unflatten operation will fail when computed!
  warnings.warn(


In [5]:
tau_ID_branches = ['tauIsVVVLoose','tauIsVVLoose', 'tauIsVLoose', 'tauIsLoose', 
                   'tauIsMedium', 'tauIsTight', 'tauIsVTight', 'tauIsVVTight']

In [7]:
signal_xSec = 1
processed_lumi = 109080
generated_signalEvents = ak.count(signal_events.evtNum).compute()
signal_normalization_factor = processed_lumi*signal_xSec/generated_signalEvents

In [18]:
def compute_Z(s, b, unc=0.2):
    '''
    Compute sensitivity for case with uncertain bkg
    From paper Soham sent
    Assumes all counts in data are bkg
    '''
    s=float(s)
    b-float(b)
    bunc = unc*b

    term1 = (s+b)*math.log(((s+b)*(b+bunc**2))/((b**2)+(s+b)*bunc**2))
    term2 = (b**2/bunc**2)*math.log(1+((bunc**2)*s)/(b*(b+bunc**2)))
    subtracted = 2*(term1-term2)
    return subtracted**0.5
    

In [19]:
# initial preselections to apply
# 1. Pass Trigger
# 2. Require exactly one reconstructed tau with pT>20GeV

mask_signal = (signal_events.HLT_CscCluster100_PNetTauhPFJet10_Loose) & (signal_events.nTaus==1) & (ak.all(signal_events.tauPt>=20, axis=1))
mask_data = (data_events.HLT_CscCluster100_PNetTauhPFJet10_Loose) & (data_events.nTaus==1) & (ak.all(data_events.tauPt>=20, axis=1))

client = Client(memory_limit="12GB", n_workers=1, 
                threads_per_worker=1, local_directory="/uscms/home/amalbert/nobackup/el9_work/CMSSW_14_1_0_pre4/src/run3_llp_analyzer/dask_temp")

denom_signal = ak.count_nonzero(mask_signal).compute()
denom_data = ak.count_nonzero(mask_data).compute()

print("denom_signal:", denom_signal)
print("denom_data:", denom_data)

masked_signal = signal_events[mask_signal]
masked_data = data_events[mask_data]

signal_eff, data_eff, signal_counts, data_counts, Z = [],[],[],[],[]
for ID in tau_ID_branches:
    print(ID)
    num_signal = ak.count_nonzero(masked_signal[ID].compute())
    signal_counts.append(num_signal*signal_normalization_factor)
    signal_eff.append(num_signal/denom_signal*100)

    num_data = ak.count_nonzero(masked_data[ID].compute())
    data_counts.append(num_data)
    data_eff.append(num_data/denom_data*100)

    Z.append(compute_Z(num_signal, num_data))


client.close()
    


/uscms/home/amalbert/.local/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42689 instead
  warnings.warn(
/uscms/home/amalbert/.local/lib/python3.9/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/cvmfs/cms.cern.ch/el8_amd64_gcc12/external/py3-numpy/1.24.3-083

denom_signal: 549
denom_data: 6432272
tauIsVVVLoose
tauIsVVLoose
tauIsVLoose
tauIsLoose
tauIsMedium
tauIsTight
tauIsVTight
tauIsVVTight


In [20]:
eff_ratio = [sig_eff/dat_eff for sig_eff, dat_eff in zip(signal_eff, data_eff)]
eff_ratio

[1.7722792877287818,
 2.6437638612064887,
 4.050475704850416,
 6.878364076569777,
 10.365273790437952,
 15.08152258808326,
 22.15326207083655,
 32.0864592272171]

In [22]:
df = pd.DataFrame({'ID WP':tau_ID_branches, "Signal Eff (%)": signal_eff,
                    "Data Eff (%)": data_eff, "Eff Ratio": eff_ratio, "Signal Counts (1pb)": signal_counts,
                    "Data Counts": data_counts, "Z": Z})
df

,ID WP,Signal Eff (%),Data Eff (%),Eff Ratio,Signal Counts (1pb),Data Counts,Z
0,tauIsVVVLoose,72.495446,40.905204,1.772279,44.196111,2631134,0.000756
1,tauIsVVLoose,67.030965,25.354369,2.643764,40.864746,1630862,0.001128
2,tauIsVLoose,62.659381,15.469635,4.050476,38.199654,995049,0.001728
3,tauIsLoose,53.369763,7.759078,6.878364,32.536333,499085,0.002935
4,tauIsMedium,44.262295,4.270249,10.365274,26.984058,274674,0.004422
5,tauIsTight,35.701275,2.367220,15.081523,21.764919,152266,0.006433
6,tauIsVTight,28.051002,1.266224,22.153262,17.101008,81447,0.009447
7,tauIsVVTight,20.765027,0.647159,32.086459,12.659188,41627,0.013676
